# Property Getters and Setters
A (very) simplified **aquarium** class to demonstrate the usage of the `property` and `<property_name>.setter` decorators.

<a title="Tom Bayly from England / CC BY-SA (https://creativecommons.org/licenses/by-sa/2.0)" href="https://commons.wikimedia.org/wiki/File:Marine_fish_(5791775758).jpg"><img width="600" alt="Marine fish (5791775758), cropped and resized" src="./aquarium.png"></a>


### Attributes
* **Salinity**:
    Measured in parts per thousands (ppt). Has been simplified to either _limninc_ (fresh water, 0.0) and _marine_ (salt water, 35.0). Although there are exceptions, for the purpose of this example it is assumed that a a fresh water fish cannot live in salt water and a salt water fish cannot live in fresh water.

* **Temperature**:
    Measured in degrees Celsius (°C). Fish can live within a specific temperature _range_ and for them to thrive and reproduce that range is even smaller.

* **Volume**:
    Measured in litres (_l_). Fish require a minimum amount of water volume to thrive. Many species also require certain minimum dimensions but we simplify by only focusing on the actual volume. We also assume that the volume is a constant (that is, the water level is always max).

* **Fish**:
    Fish of the `Fish` class that have been added to the aquarium. There are multiple factors, a lot more than the three above, that determine whether a fish can thrive in an aquarium. But for the purpose of this example we focus only on _salinity_, _temperature_ and _volume_.


In [3]:
from typing import Sequence
from enum import Enum

In [4]:
class Salinity(Enum):
    """Salinity in ppt (‰)."""
    limnic: float = 0.0
    marine: float = 35.0

In [5]:
class MetaFishCounter(type):
    @property
    def count(cls) -> int:
        """Count number of fish by species.
        
        Return sum of all fish if class is Fish.
        """
        if cls == Fish:
            return cls._total
        return cls._count

    @property
    def total(cls) -> int:
        """Count total number of fish.
        
        Return sum of all fish of species if class is not Fish.
        """
        if cls == Fish:
            return cls._total
        return cls._count


# Todo: ABC?
class Fish(metaclass=MetaFishCounter):
    common_name: str
    salinity: Salinity
    min_temperature_c: float
    max_temperature_c: float
    min_volume_l: float
    _count: int = 0
    _total: int = 0

    def __init__(self):
        """Set unique id for each instantiated fish to differentiate between
          individuals.
        """
        Fish._total += 1
        type(self)._count += 1
        self._count: int = type(self)._count

    @property
    def count(self) -> int:
        return type(self).count

    @property
    def total(self) -> int:
        return type(self).count

    @property
    def ID(self):
        return (f"{''.join(word for word in self.common_name.lower().split())}-" 
                f'{str(self._count).zfill(3)}')

    def __repr__(self) -> str:
        return f"""<Fish {self.common_name} #{self.ID}>
Min required volume (l): {self.min_volume_l}
Min temperature (°C): {self.min_temperature_c}
Max temperature (°C): {self.max_temperature_c}
Salinity: {self.salinity.name} ({self.salinity.value}‰)
"""


class BlueTang(Fish):
    id_prefix: str = 'bt'
    common_name = 'Blue Tang'
    salinity = Salinity.marine
    min_temperature_c = 22.23
    max_temperature_c = 25.6
    min_volume_l = 378.6


class YellowTang(Fish):
    id_prefix: str = 'yt'
    common_name = 'Yellow Tang'
    salinity = Salinity.marine
    min_temperature_c = 22.23
    max_temperature_c = 25.6
    min_volume_l = 378.6


class Blacktip(Fish):
    id_prefix: str = 'bts'
    common_name = 'Blacktip Reef Shark'
    salinity = Salinity.marine
    min_temperature_c = 22.23
    max_temperature_c = 25.6
    min_volume_l = 37854.11784


class Betta(Fish):
    id_prefix: str = 'ba'
    common_name='Siamese Fighting Fish'
    salinity = Salinity.limnic
    min_temperature_c = 24.0
    max_temperature_c = 28.0
    min_volume_l = 35.0

In [6]:
class Aquarium:
    """Represents an aquarium."""

    def __init__(
        self,
        volume_l: float,
        salinity: Salinity,
        temperature_c: float,
    ) -> None:
        """Define aquarium size and water values. Fish are added later.
        
        All attributes are 'protected/private', meaning that they should not be
          accessed or modified outside of the class.

        To set and get the attributes we use property decorators, to avoid changing
          a variable that will harm any fish already in the aquarium, or add fish that
          wouldn't thrive in the current environment.
        """
        self._VOLUME: float = volume_l
        self._salinity: Salinity = salinity
        self._temperature: float = temperature_c
        self._fish: Sequencequence[Fish] = []

    def __repr__(self) -> str:
        return (f'<Aquarium {self.VOLUME}l, {self.temperature}°C,'
               f'{self.salinity.name} ({self.salinity.value}‰)>')

    @property
    def VOLUME(self) -> float:
        """Volume in litres.
        It is assumed the water level always is at the max level, making this value a constant
          since the dimensions cannot be changed.
        """
        return self._VOLUME

    def has_valid_volume(self, fish: Fish) -> bool:
        """Check if volume is viable for fish being introduced to the aquarium.
        
        Since volume is a constant, this method is only used when adding fish to the aquarium.
          Thus it can safely use self rather than being a static method.
        """
        if fish.min_volume_l > self.VOLUME:
            return False
        return True

    @property
    def salinity(self) -> Salinity:
        """Salinity category.
        This value is mutable but cannot be changed without being evaluated and suitable
          for all fish in the aquarium.
        """
        return self._salinity

    @salinity.setter
    def salinity(self, salinity: Salinity) -> None:
        """Set new salinity only if it's viable for all fish in the aquarium."""
        if Aquarium.is_valid_salinity(salinity, self.fish):
            self._salinity = salinity
        else:
            raise ValueError("Unable to set salinity, it's not viable for all fish in the aquarium.")

    @staticmethod
    def is_valid_salinity(salinity: Salinity, fishes: Sequence[Fish]) -> bool:
        """Check if new salinity is viable for each fish currently inhabiting or
          being introduced to the aquarium.
        
        If there is no fish, temperature can always be changed.
        We could use a list comprehension here but a for loop enables us to make an early return.
        """
        for fish in fishes:
            if fish.salinity is not salinity:
                return False
        return True

    @property
    def temperature(self) -> float:
        """Temperature in degrees Celsius."""
        return self._temperature

    @temperature.setter
    def temperature(self, temperature) -> None:
        """Set new salinity only if it's viable for all fish in the aquarium."""
        if Aquarium.is_valid_temperature(temperature, self.fish):
            self._salinity = salinity
        else:
            raise ValueError("Unable to set salinity, it's not viable for all fish in the aquarium.")

    @staticmethod
    def is_valid_temperature(temperature: float, fishes: Sequence[Fish]) -> bool:
        """Check if new temperature is viable for each fish currently inhabiting or
          being introduced to the aquarium.
        
        To determine each fish's temperature range we use comparison operator chaining, including
          both the min and max temperature values.
        If there is no fish, temperature can always be changed.
        We could use a list comprehension here but a for loop enables us to make an early return.
        """
        for fish in fishes:
            if not fish.min_temperature_c <= temperature <= fish.max_temperature_c:
                return False
        return True

    @property
    def fish(self) -> Sequence[Fish]:
        """Fish currently inhabiting the aquarium."""
        return self._fish

    @fish.setter
    def fish(self, fish: Fish) -> None:
        """Add fish to the aquarium.

        For the purpose of this example fish can only be added and not removed.
        """
        # Check if aquarium volume is sufficient.
        if not self.has_valid_volume(fish):
            raise ValueError(f'Unable to add fish {fish.common_name}; '
                             f'Volume is too small ({self.VOLUME} l), requires at least {fish.min_volume_l} l')

        # Check if water properties are viable.
        # We use a tuple here to satisfy the type of the method argument.
        if not Aquarium.is_valid_salinity(self.salinity, (fish, )):
            raise ValueError(f'Unable to add fish {fish.common_name}; '
                             f'Invalid salinity ({self.salinity.name}, {self.salinity.value}‰)')

        if not Aquarium.is_valid_temperature(self.temperature, (fish, )):
            raise ValueError(f'Unable to add fish {fish.common_name}; '
                             f'Invalid temperature ({self.temperature}°C)')

        self._fish.append(fish)


In [7]:
salt_water_600: Aquarium = Aquarium(volume_l=600.00, temperature_c=25.5, salinity=Salinity.marine)
salt_water_600

<Aquarium 600.0l, 25.5°C,marine (35.0‰)>

In [8]:
yellow_tang: YellowTang = YellowTang()
yellow_tang

<Fish Yellow Tang #yellowtang-001>
Min required volume (l): 378.6
Min temperature (°C): 22.23
Max temperature (°C): 25.6
Salinity: marine (35.0‰)

In [9]:
salt_water_600.fish = yellow_tang

In [10]:
salt_water_600.fish

[<Fish Yellow Tang #yellowtang-001>
 Min required volume (l): 378.6
 Min temperature (°C): 22.23
 Max temperature (°C): 25.6
 Salinity: marine (35.0‰)]

In [11]:
blacktip: Blacktip = Blacktip()
blacktip

<Fish Blacktip Reef Shark #blacktipreefshark-001>
Min required volume (l): 37854.11784
Min temperature (°C): 22.23
Max temperature (°C): 25.6
Salinity: marine (35.0‰)

In [12]:
salt_water_600.fish = blacktip

ValueError: Unable to add fish Blacktip Reef Shark; Volume is too small (600.0 l), requires at least 37854.11784 l

In [13]:
betta: Betta = Betta()
betta

<Fish Siamese Fighting Fish #siamesefightingfish-001>
Min required volume (l): 35.0
Min temperature (°C): 24.0
Max temperature (°C): 28.0
Salinity: limnic (0.0‰)

In [14]:
salt_water_600.fish = betta

ValueError: Unable to add fish Siamese Fighting Fish; Invalid salinity (marine, 35.0‰)

In [15]:
salt_water_600.fish = YellowTang()
blue_tang = BlueTang()
blue_tang

<Fish Blue Tang #bluetang-001>
Min required volume (l): 378.6
Min temperature (°C): 22.23
Max temperature (°C): 25.6
Salinity: marine (35.0‰)

In [16]:
for _ in range(6):
    salt_water_600.fish = BlueTang()

In [17]:
for fish in salt_water_600.fish:
    print(fish.common_name, f'#{fish.ID}')

Yellow Tang #yellowtang-001
Yellow Tang #yellowtang-002
Blue Tang #bluetang-002
Blue Tang #bluetang-003
Blue Tang #bluetang-004
Blue Tang #bluetang-005
Blue Tang #bluetang-006
Blue Tang #bluetang-007


In [20]:
BlueTang.count, YellowTang.count, Blacktip.count, Betta.count

(7, 2, 1, 1)

In [21]:
Fish.count

11

In [23]:
len(salt_water_600.fish)

8